In [14]:
install.packages("Rcpp")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [16]:
library(Rcpp)

#### 1. Calculate Squared Distance

$D = dist2(X, C)$ takes two matrices of vectors and calculates the squared Euclidean distance between them.  Both matrices must be of the same column dimension.  If $X$ has $M$ rows and $N$ columns, and $C$ has $L$ rows and $N$ columns, then the result has $M$ rows and $L$ columns.  The $i$, $j$th entry is the  squared distance from the $i$th row of $X$ to the $j$th row of $C$.

In [1]:
dist2 <- function(x, c) {
    # Get the number of data points and dimensions in x
    ndata <- nrow(x)
    dimx <- ncol(x)
    
    # Get the number of centres and dimensions in c
    ncentres <- nrow(c)
    dimc <- ncol(c)
    
    # Check that the dimensions of x and c match
    if (dimx != dimc) {
        stop("Data dimension does not match dimension of centres")
    }
    
    # Calculate the squared Euclidean distance between each data point in x and each centre in c
    n2 <- matrix(0, nrow = ndata, ncol = ncentres)
    for (i in 1:ncentres) {
        n2[, i] <- colSums((t(x) - t(c)[,i])^2)
    }
    
    #n2 <- matrix(rowSums(x^2), ndata, 1) %*% matrix(rep(1, ncentres), ncol = ncentres) + 
    #      matrix(rep(rowSums(c), ndata), ncol = ncentres, byrow = TRUE) -
    #      2 * x %*% t(c)
    
    # Return the squared Euclidean distance matrix
    return(n2)
}

In [2]:
#install.packages('fields')

In [4]:
set.seed(1)
x <- matrix(rnorm(2*3, mean=2, sd=2), nrow=2)
c <- matrix(runif(5*3), nrow=5)

# Use r built-in distance function to check if the above calculation is correct
library(fields)
print(paste0('Number of different elements from two distance matrices calculated is ', sum((round(dist2(x, c),2) == round(rdist(x,c)^2, 2)) == FALSE)))

[1] "Number of different elements from two distance matrices calculated is 0"


#### 2. Cost Function

In [5]:
hist_cost_2 <- function(BH1, BH2, eps=0.001) {
    # same as hist_cost.m but BH1 and BH2 can be of different lengths
    nsamp1 <- dim(BH1)[1]
    nbins <- dim(BH1)[2]
    nsamp2 <- dim(BH2)[1]
    
    BH1n <- BH1 / matrix(rep(rowSums(BH1)+eps, nbins), nrow=nsamp1, byrow=F)
    BH2n <- BH2 / matrix(rep(rowSums(BH2)+eps, nbins), nrow=nsamp2, byrow=F)
    
    tmp1 <- array(rep(BH1n, each=nsamp2), dim=c(nsamp1, nsamp2, nbins))
    tmp2 <- array(rep(t(BH2n), each=nsamp1), dim=c(nsamp1, nsamp2, nbins))
    
    HC <- 0.5 * apply((tmp1 - tmp2)^2 / (tmp1 + tmp2 + eps), c(1, 2), sum)
    return(HC)
}

#### 3. Sample Sequence from Designated Points

In [6]:
# Sample sequence from designated points
# Given: the sequence of feature points
# Return: subsequences
samplingSequencesIdx <- function(sensordata, seqlen, idx) {
  # Check if the correct number of arguments have been provided
  if(missing(idx)) {
    stop("Insufficient number of arguments")
  }
  
  # Get the number of timestamps and dimensions
  nTimeStamps <- nrow(sensordata)
  nDims <- ncol(sensordata)
  
  # Check if the input matrix is in the correct format
  if(nTimeStamps < nDims) {
    warning("Synchronized signals should be arranged column-wisely\n")
  }
  
  # Initialize variables
  mask <- 1:seqlen
  mask <- mask - ceiling(seqlen/2)
  nSeqs <- length(idx)
  sequences <- list()
  tAnchor <- idx
  cnt <- 0
  
  # Loop through each subsequence
  for(i in 1:nSeqs) {
    iloc <- idx[i]
    imask <- mask + iloc
    
    # Check if the subsequence is within the bounds of the input matrix
    if(imask[1] <= 0 && imask[length(imask)] <= nTimeStamps) {
      tmp <- matrix(rep(sensordata[1, ], (1-imask[1])*nDims), ncol=nDims, byrow=TRUE)
      isequence <- rbind(tmp, sensordata[1:imask[length(imask)], ])
    } 
    else if(imask[length(imask)] > nTimeStamps && imask[1] >= 1) {
      tmp <- matrix(rep(sensordata[nTimeStamps, ], (imask[length(imask)]-nTimeStamps)*nDims), ncol=nDims, byrow=TRUE)
      isequence <- rbind(sensordata[imask[1]:nTimeStamps, ], tmp)
    } 
    else if(imask[1] <= 0 && imask[length(imask)] > nTimeStamps) {
      tmp1 <- matrix(rep(sensordata[1, ], (1-imask[1])*nDims), ncol=nDims, byrow=TRUE)
      tmp2 <- matrix(rep(sensordata[nTimeStamps, ], (imask[length(imask)]-nTimeStamps)*nDims), ncol=nDims, byrow=TRUE)
      isequence <- rbind(tmp1, sensordata, tmp2)
    } 
    else {
      isequence <- sensordata[imask, ]
    }
    
    cnt <- cnt + 1
    tAnchor[cnt] <- iloc
    sequences <- rbind(sequences, list(isequence))
  }
  
  tAnchor <- tAnchor[1:cnt]
  
  return(list(sequences, tAnchor))
}

In [7]:
# Read data from file
data_urc <- read.table('data/dataSet_URC.txt', quote="\"", comment.char="")[,2:10]
samplingSequencesIdx(data_urc, 5, c(3:5))

"0.4248307, 0.6525383, 0.4046725, 1.0870582, 0.4436209, 1.4173710, 2.1728969, 0.5536035, 2.0101909, 0.9472850, 2.6664697, 3.6374980, 0.7235771, 2.2997473, 1.9240844, 3.2957564, 2.2746095, 1.4479759, 1.6287423, 2.1597897, 2.2625860, 0.9763371, 2.0111302, 0.6360933, 1.4994468, 0.1650483, -0.3876330, 1.6232818, -0.1405643, 0.9645078, -0.9588735, -0.9102164, 1.2038459, 0.5473488, 0.2232562, 0.2824344, -0.1481396, 1.1166959, 1.0740507, -0.4249456, 0.8421263, 0.3047345, 0.5617010, 0.8959816, 0.1212787"
"0.652538270, 0.404672490, 1.087058200, 0.443620910, 0.209288520, 2.172896900, 0.553603530, 2.010190900, 0.947285030, 1.272233300, 3.637498000, 0.723577080, 2.299747300, 1.924084400, 2.821823100, 2.274609500, 1.447975900, 1.628742300, 2.159789700, 3.589123700, 0.976337080, 2.011130200, 0.636093330, 1.499446800, 2.221711100, -0.387633030, 1.623281800, -0.140564330, 0.964507760, 0.569911390, -0.910216450, 1.203845900, 0.547348750, 0.223256230, -0.937877270, -0.148139590, 1.116695900, 1.074050700, -0.424945600, -0.812020570, 0.304734520, 0.561700960, 0.895981570, 0.121278670, -0.009299545"
"0.404672490, 1.087058200, 0.443620910, 0.209288520, 1.084084500, 0.553603530, 2.010190900, 0.947285030, 1.272233300, 1.852098600, 0.723577080, 2.299747300, 1.924084400, 2.821823100, 1.832584200, 1.447975900, 1.628742300, 2.159789700, 3.589123700, 1.548627700, 2.011130200, 0.636093330, 1.499446800, 2.221711100, 1.381758000, 1.623281800, -0.140564330, 0.964507760, 0.569911390, 1.000645300, 1.203845900, 0.547348750, 0.223256230, -0.937877270, 0.254974440, 1.116695900, 1.074050700, -0.424945600, -0.812020570, -0.332783530, 0.561700960, 0.895981570, 0.121278670, -0.009299545, -0.444037440"


#### 4. Dynamic Programming

In [8]:
# [p,q,D,sc] = dpfast(M,C,T,G) 
#    Use dynamic programming to find a min-cost path through matrix M.
#    Return state sequence in p,q; full min cost matrix as D and 
#    local costs along best path in sc.
#    This version gives the same results as dp.m, but uses dpcore.mex
#    to run ~200x faster.
#    C is a step matrix, with rows (i step, j step, cost factor)
#    Default is [1 1 1.0;0 1 1.0;1 0 1.0];
#    Another good one is [1 1 1;1 0 1;0 1 1;1 2 2;2 1 2]
#    T selects traceback origin: 0 is to any edge; 1 is top right (default);
#    T > 1 finds path to min of anti-diagonal T points away from top-right.
#    Optional G defines length of 'gulleys' for T=0 mode; default 0.5
#    (i.e. accept path to only 50% of edge nearest top-right)
# 2003-04-04,2005-04-04 dpwe@ee.columbia.edu $Header: /Users/dpwe/projects/dtw/RCS/dpfast.m,v 1.6 2008/03/14 14:40:50 dpwe Exp dpwe $
  
  # Copyright (c) 2003 Dan Ellis <dpwe@ee.columbia.edu>
  # released under GPL - see file COPYRIGHT

dpfast <- function(M, C=c(1,1,1.0), T=1, G=0.5) {
    if (missing(M)) {
        stop("Error: M is missing")
    }
    
    # Default step / cost matrix
    if (missing(C)) {
        C = matrix(c(1, 1, 1.0, 0, 1, 1.0, 1, 0, 1.0), ncol=3, byrow=TRUE)
    }
    
    # Default: path to top-right
    if (missing(T)) {
        T = 1
    }
    
    # how big are gulleys?
    if (missing(G)) {
        G = 0.5  # half the extent
    }
    
    if (sum(is.nan(M)) > 0) {
        stop("Error: Cost matrix includes NaNs")
    }
    
    if (min(M) < 0) {
    warning("Warning: cost matrix includes negative values; results may not be what you expect")
    }
    
    r <- nrow(M)
    c <- ncol(M)
    
    # Core cumulative cost calculation coded as mex
    D_phi <- dpcore(M, C)
    D <- D_phi$D
    phi <- D_phi$phi
    
    p <- NULL
    q <- NULL
    
    ## Traceback from top left?
    #i <- r
    #j <- c
    if (T == 0) {
        # Traceback from lowest cost "to edge" (gulleys)
        TE <- D[r,]
        RE <- D[,c]
        # eliminate points not in gulleys
        TE[1:round((1-G)*c)] <- max(D)
        RE[1:round((1-G)*r)] <- max(D)
        if (min(TE) < min(RE)) {
            i <- r
            j <- max(which(TE==min(TE)))
        } else {
            i <- max(which(RE==min(RE)))
            j <- c
        }
    } else {
        if (min(dim(D)) == 1) {
            # degenerate D has only one row or one column - messes up diag
            i <- r
            j <- c
        } else {
            # Traceback from min of antidiagonal
            #stepback = floor(0.1*c)
            stepback <- T
            slice <- diag(flip(D), -(r-stepback))
            ii <- which.min(slice)
            i <- r - stepback + ii
            j <- c + 1 - ii
        }
    }
    
    p <- i
    q <- j
    sc <- M[p, q]
    
    while (i >= 1 && j >= 1) {
        if (i == 1 && j == 1) {
            break
        }
        #disp(paste0("i=",i," j=",j))
        tb <- phi[i, j]
        i <- i - C[tb, 1]
        j <- j - C[tb, 2]
        p <- c(i, p)
        q <- c(j, q)
        sc <- c(M[i, j], sc)
    }
    
    return(list(p=p, q=q, D=D, sc=sc))
}

#### 5. Shape Descriptors

In [9]:
calcDescriptor <- function(subsequence, descriptorName = "HOG1D", param = NULL) {
  if (length(dim(subsequence)) != 1) {
    stop("The input subsequence should be one-dimensional\n")
  }
  
  if (missing(param)) {
    param <- validateHOG1Dparam()
  }
  
  switch(descriptorName,
         "HOG1D" = {
           val_param <- validateHOG1Dparam(param)
           descriptor <- descriptorHOG1D(subsequence, val_param)
         },
         "PAA" = {
           val_param <- validatePAAdescriptorparam(param)
           descriptor <- descriptorPAA(subsequence, val_param)
         },
         "DWT" = {
           val_param <- validateDWTdescriptorparam(param)
           descriptor <- descriptorDWT(subsequence, val_param)
         },
         stop("Only support 3 descriptors\n")
  )
  
  return(descriptor)
}

#### 6. Validate Descriptor

In [10]:
validateDWTdescriptorparam <- function(param) {
    if (missing(param) || !is.list(param)) {
        val_param <- list(numLevels = 2, selection = 'Haar', n = 2)
        return(val_param)
    }
    
    val_param <- param
    if (!'numLevels' %in% names(param)) {
        val_param$numLevels <- 2
    }
    if (!'selection' %in% names(param)) {
        val_param$selection <- 'Haar'
    }
    
    if (!'n' %in% names(param)) {
        val_param$n <- 2
    }
    return(val_param)
}

In [11]:
validateHOG1Dparam <- function(param = NULL) {
  # a typical valid setting
  # {
  #   param.blocks    = [1 2];  # unit: cell
  #   param.cells     = [1 25]; # unit: t
  #   param.gradmethod = 'centered'; # 'centered'
  #   param.nbins = 8;
  #   param.sign = 'true';
  # }
  if (is.null(param) || !is.list(param)) {
    val_param <- list(
      # blocks = c(1, 2),
      cells = c(1, 25),
      overlap = 0,
      gradmethod = "centered",
      nbins = 8,
      sign = "true",
      xscale = 0.1
      # cutoff = c(2, 51)
    )
    return(val_param)
  }
  
  val_param <- param
  
  if (!("overlap" %in% names(val_param))) {
    val_param$overlap <- 12
  }
  
  if (!("xscale" %in% names(val_param))) {
    val_param$xscale <- 0.1
  }
  
  if (!("cells" %in% names(val_param))) {
    val_param$cells <- c(1, 25)
  }
  
  if (!("gradmethod" %in% names(val_param))) {
    val_param$gradmethod <- "uncentered"
  }
  
  if (!("nbins" %in% names(val_param))) {
    val_param$nbins <- 8
  }
  
  if (!("sign" %in% names(val_param))) {
    val_param$sign <- "true"
  }
  
  return(val_param)
}

In [12]:
# validate parameters of piecewise aggregate approximation
validatePAAparam <- function(param) {
  
  if (missing(param) || !is.list(param)) {
    val_param <- list(segNum = 10, segLen = NULL, priority = "segNum")
    return(val_param)
  }
  
  val_param <- list()
  
  if (!is.null(param$segNum)) {
    if (param$segNum > 0) {
      val_param$segNum <- param$segNum
    } else {
      val_param$segNum <- 10
    }
  } else {
    val_param$segNum <- 10
  }
  
  if (!is.null(param$segLen)) {
    if (param$segLen > 0) {
      val_param$segLen <- param$segLen
    } else {
      val_param$segLen <- 5
    }
  } else {
    val_param$segLen <- 5
  }
  
  if (!is.null(param$priority)) {
    val_param$priority <- param$priority
  } else {
    val_param$priority <- "segNum"
  }
  
  switch(val_param$priority,
         segNum = {
           if (is.null(val_param$segNum) || val_param$segNum <= 0) {
             val_param$segNum <- 10
           }
         },
         segLen = {
           if (is.null(val_param$segLen) || val_param$segLen <= 0) {
             val_param$segLen <- 5
           }
         },
         {
           val_param$priority <- "segNum"
           if (is.null(val_param$segNum) || val_param$segNum <= 0) {
             val_param$segNum <- 10
           }
         }
  )
  
  return(val_param)
}

validatePAAdescriptorparam <- function(param = NULL) {
  
  if (is.null(param) || !is.list(param)) {
    val_param <- list(segNum = 10, segLen = NULL, priority = "segNum")
    return(val_param)
  }
  
  val_param <- validatePAAparam(param)
  return(val_param)
  
}

#### 7. ShapeDTW

In [13]:
shapeDTW <- function(p, q, seqlen, descriptorSetting = NULL, metric = "Euclidean") {
  if (!is.vector(p) || !is.vector(q)) {
    stop("Only support univariate time series\n")
  }
  
  if (is.null(metric)) {
    metric <- "Euclidean"
  }
  
  p <- p %>% t() %>% as.data.frame()
  q <- q %>% t() %>% as.data.frame()
  
  lenp <- length(p)
  lenq <- length(q)
  
  if (is.null(descriptorSetting)) {
    hog <- validateHOG1Dparam()
    hog$cells <- c(1, round(seqlen/2)-1)
    hog$overlap <- 0
    hog$xscale <- 0.1
    
    paa <- validatePAAdescriptorparam()
    paa$priority <- "segNum"
    segNum <- ceiling(seqlen/5)
    paa$segNum <- segNum
    
    numLevels <- 3
    dwt <- validateDWTdescriptorparam()
    dwt$numLevels <- numLevels
    
    self <- NULL
    
    descriptorSetting <- list(method = "HOG1D", param = hog)
  }
  
  # First compute descriptor at each point, and transform the univariate time series to multivariate one
  p_subsequences <- samplingSequencesIdx(p, seqlen, 1:lenp)
  q_subsequences <- samplingSequencesIdx(q, seqlen, 1:lenq)
  
  # Descriptor
  descriptorName <- descriptorSetting$method
  descriptorParam <- descriptorSetting$param
  
  p_nsubsequences <- length(p_subsequences)
  p_descriptors <- vector("list", p_nsubsequences)
  for (j in 1:p_nsubsequences) {
    seq <- p_subsequences[[j]]
    p_descriptors[[j]] <- calcDescriptor(seq, descriptorName, descriptorParam)
  }
  p_descriptors <- do.call(rbind, p_descriptors)
  
  q_nsubsequences <- length(q_subsequences)
  q_descriptors <- vector("list", q_nsubsequences)
  for (j in 1:q_nsubsequences) {
    seq <- q_subsequences[[j]]
    q_descriptors[[j]] <- calcDescriptor(seq, descriptorName, descriptorParam)
  }
  q_descriptors <- do.call(rbind, q_descriptors)
  
  # Match multivariate time series 'p_descriptors' & 'q_descriptors'
  # Run DTW
  switch(metric,
         "Euclidean" = {
           d <- dist2(p_descriptors, q_descriptors)
           d <- sqrt(d)
         },
         "chi-square" = {
           d <- hist_cost_2(p_descriptors, q_descriptors)
         },
         stop("Only support two distance metrics\n")
  )
  
  dtw_res <- dpfast(d)
  
  idxp <- dtw_res[[1]]
  idxq <- dtw_res[[2]]
  cD <- dtw_res[[3]]
  pc <- dtw_res[[4]]
  
  match <- cbind(idxp, idxq)
  lPath <- length(idxp)
  distDescriptor <- sum(pc)
  
  # Compute distance using raw signals, instead of descriptor distances
  wp <- wpath2mat(idxp) %*% p
  wq <- wpath2mat(idxq) %*% q
  
  distRaw <- sum(sqrt((wp-wq)^2))
  
  return(list(distRaw = distRaw, distDescriptor = distDescriptor))
}